In this notebook, I have tried to create a pipeline model for imputing missing data and doing Label encoding. To reduce the work further you can create the function. This is still work in progrss. I haven't added much markdown becuase each cell is self explaintory. 

In the coming version, I will be adding more graps visualizations and we will try to incease the precison of the model.

Importing the Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("../input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
#convert 'Credit_History' as an object because it is cointaing just 0 and 1
data['Credit_History'] = data['Credit_History'].astype('O')

In [ ]:
data.describe()

In [ ]:
data.describe(include='O')

In [ ]:
X = data.drop(labels=['Loan_ID','Loan_Status'], axis=1)
X

In [ ]:
Y = data['Loan_Status']
Y 

In [ ]:
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
X.shape, Y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
X_train.info()

In [ ]:
X_train.isnull().mean().sort_values(ascending=False)

In [ ]:
pd.DataFrame(y_train).isnull().mean()

Features are replaced as belwo rule:
   1. <= 5% and categorlcal: with most frequent data
   2. > 5% and categoriacl : with word "Missing"
   3. < 5% and numerical: with mean

In [ ]:
features_numeric = ['Loan_Amount_Term','LoanAmount','CoapplicantIncome','ApplicantIncome']
features_categoric_one = ['Gender','Married','Dependents','Property_Area','Education']
features_categoric_two = ['Credit_History','Self_Employed']


In [ ]:
#for mising data more than 5% we will replace nan value with a word "Missing"
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
 imputer_numeric = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
])

imputer_categoric_one = Pipeline(
    steps=[('imputer',
            SimpleImputer(strategy='most_frequent'))])

imputer_categoric_two = Pipeline(
    steps=[('imputer',
            SimpleImputer(strategy='constant', fill_value='Missing'))])

# then we put the features list and the transformers together
# using the column transformer
preprocessor = ColumnTransformer(transformers=[
    ('imputer_numeric',imputer_numeric,features_numeric),
    ('imputer_categoric_one',imputer_categoric_one,features_categoric_one),
    ('imputer_categoric_two',imputer_categoric_two,features_categoric_two)
])

In [ ]:
preprocessor.fit(X_train)

In [ ]:
 preprocessor.transformers

In [ ]:
# and we can look at the parameters learnt like this:

# for the numerical imputer
preprocessor.named_transformers_['imputer_numeric'].named_steps['imputer'].statistics_

In [ ]:
preprocessor.named_transformers_['imputer_categoric_one'].named_steps['imputer'].statistics_

In [ ]:
preprocessor.named_transformers_['imputer_categoric_two'].named_steps['imputer'].statistics_

In [ ]:
#First Transformation
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
X_train =  pd.DataFrame(X_train, columns=features_numeric+features_categoric_one+features_categoric_two)
X_test =  pd.DataFrame(X_test, columns=features_numeric+features_categoric_one+features_categoric_two) 

In [ ]:
X_train.isnull().mean()

In [ ]:
X_train['LoanAmount'].nunique()

In [ ]:
X_test['Loan_Amount_Term'].unique()
X_test_cp = X_test[X_test['Loan_Amount_Term'] != 12.0 ]

In [ ]:
X_test_cp['LoanAmount'].nunique()

In [ ]:
X_test.isnull().mean()

In [ ]:
 X_train.shape, X_test.shape

In [ ]:
 for col in features_categoric_one+features_categoric_two:
        print(X_train[col].value_counts())
        print("--------------------------")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train['Credit_History'] = X_train['Credit_History'].astype(str)
X_test['Credit_History'] = X_test['Credit_History'].astype(str)

In [ ]:
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [4,5,6,7,8,9,10])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

models = {
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=1, random_state=42)
}

In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, accuracy_score

def loss(y_true, y_pred, retu=False):
    pre = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    loss = log_loss(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    
    if retu:
        return pre, rec, f1, loss, acc
    else:
        print('  pre: %.3f\n  rec: %.3f\n  f1: %.3f\n  loss: %.3f\n  acc: %.3f' % (pre, rec, f1, loss, acc))

In [ ]:
# train_eval_train

def train_eval_train(models, X, y):
    for name, model in models.items():
        print(name,':')
        model.fit(X, y)
        loss(y, model.predict(X))
        print('-'*40)
        
train_eval_train(models, X_train, y_train)

# we can see that best model is LogisticRegression at least for now, 
#SVC is just memorizing the data so it is overfitting .

In [ ]:
for name, model in models.items():
        print(name,':')
        loss(y_test, model.predict(X_test))
        print("------------------------------")

In [ ]:
#since we are getting maximum accuraccy with Decision tree, we will go ahead with Decison tree,
#reading submission data
data_sub = pd.read_csv("../input/loan-prediction-problem-dataset/test_Y3wMUE5_7gLdaTN.csv")

In [ ]:
data_sub.head()

In [ ]:
data_sub.isnull().mean().sort_values(ascending=False)

In [ ]:
loan_id = data_sub["Loan_ID"]
data_sub = data_sub.drop(labels=["Loan_ID"], axis =1)
data['Credit_History'] = data['Credit_History'].astype('O')

removing missing data


In [ ]:
data_sub = preprocessor.transform(data_sub)

In [ ]:
data_sub =  pd.DataFrame(data_sub, columns=features_numeric+features_categoric_one+features_categoric_two)

In [ ]:
data_sub.isnull().mean()

In [ ]:
data_sub['Credit_History'] = data_sub['Credit_History'].astype(str)

In [ ]:
#doing label encoding
for col in data_sub.columns:
    print(data_sub[col].dtype)

In [ ]:
data_sub = ct.transform(data_sub.fillna(method='bfill'))

In [ ]:
final_pred = models['DecisionTreeClassifier'].predict(data_sub)

In [ ]:
loan_id

In [ ]:
d = pd.DataFrame([loan_id.astype(str), final_pred.astype(str)])
d = d.transpose()

In [ ]:
d.to_csv('submission1.csv', index=False)